## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
import tensorflow as tf

In [11]:
tf.__version__

'2.0.0'

In [0]:
!pip install -U tensorflow==2.0 --quiet

In [3]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default
import tensorflow as tf
tf.__version__

'2.0.0'

### Collect Data

In [9]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/gdrive/My Drive/GreatlakesAIML/Neural Networks Notebooks/Lab 3 nov/prices.csv')

### Check all columns in the dataset

In [14]:
data.shape

(851264, 7)

In [15]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [0]:
data = data.drop('date', axis=1)

In [0]:
### Drop columns `date` and  `symbol`
data = data.drop('symbol', axis=1)

In [20]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


[link text](https://)### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data = data.iloc[:1000] 

In [22]:
data.shape

(1000, 5)

In [0]:
data['volume']=data['volume'].div(1000000)

In [24]:
data

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086
...,...,...,...,...,...
995,63.310001,63.590000,63.240002,63.639999,2.1332
996,27.160000,26.990000,26.680000,27.299999,1.9824
997,28.320000,28.770000,28.010000,28.809999,37.1528
998,44.000000,44.799999,43.750000,44.810001,6.5686


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
X = data[data.columns[1:-1]]
y = data["volume"]

In [28]:
X

,close,low,high
0,125.839996,122.309998,126.250000
1,119.980003,119.940002,125.540001
2,114.949997,114.930000,119.739998
3,116.620003,113.500000,117.440002
4,114.970001,114.089996,117.330002
...,...,...,...
995,63.590000,63.240002,63.639999
996,26.990000,26.680000,27.299999
997,28.770000,28.010000,28.809999
998,44.799999,43.750000,44.810001


In [0]:
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.30, random_state=1)

In [31]:
type(train_x)

pandas.core.frame.DataFrame

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
import numpy as np

In [0]:
train_x=np.array(train_x.astype('float32'))

In [0]:
train_y=np.array(train_y.astype('float32'))

In [0]:
test_x=np.array(test_x.astype('float32'))

In [0]:
test_y=np.array(test_y.astype('float32'))

In [39]:
type(train_x)

numpy.ndarray

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

In [0]:
transformer = Normalizer()
train_x = transformer.fit_transform(train_x)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
w = tf.zeros(shape=(3,1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w) #tf.matmul will run in tensor flow and np.matmul can run in numpy in CPU also. 
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr) # This is average of all losses. We get single value for loss
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [67]:
train_x.shape

(700, 3)

In [68]:
for i in range(100):
    
    w, b = train(train_x, train_y, w, b)
    print('Current Loss on iteration', i, loss(train_y, prediction(train_x, w, b)).numpy())

Current Loss on iteration 0 223.49037
Current Loss on iteration 1 221.28824
Current Loss on iteration 2 219.25897
Current Loss on iteration 3 217.38873
Current Loss on iteration 4 215.66515
Current Loss on iteration 5 214.0766
Current Loss on iteration 6 212.61264
Current Loss on iteration 7 211.26344
Current Loss on iteration 8 210.02008
Current Loss on iteration 9 208.87431
Current Loss on iteration 10 207.81819
Current Loss on iteration 11 206.84474
Current Loss on iteration 12 205.94778
Current Loss on iteration 13 205.12117
Current Loss on iteration 14 204.35931
Current Loss on iteration 15 203.6573
Current Loss on iteration 16 203.01022
Current Loss on iteration 17 202.41382
Current Loss on iteration 18 201.86414
Current Loss on iteration 19 201.35771
Current Loss on iteration 20 200.89087
Current Loss on iteration 21 200.46071
Current Loss on iteration 22 200.0642
Current Loss on iteration 23 199.6988
Current Loss on iteration 24 199.36212
Current Loss on iteration 25 199.05167


### Get the shapes and values of W and b

In [69]:
#Check Weights and Bias
print('Weights:\n', w.numpy())
print('Bias:\n',b.numpy())

Weights:
 [[1.5694233]
 [1.5478165]
 [1.5828029]]
Bias:
 [2.7137406]


In [70]:
train_x[0]

array([0.5776868, 0.5726725, 0.5816564], dtype=float32)

### Model Prediction on 1st Examples in Test Dataset

In [93]:
predict = prediction(train_x[0:1],w,b)
predict

<tf.Tensor: id=23531, shape=(1, 1), dtype=float32, numpy=array([[5.427415]], dtype=float32)>

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
data = pd.read_csv('/gdrive/My Drive/GreatlakesAIML/Neural Networks Notebooks/Lab 3 nov/Iris.csv')

In [88]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
Id               150 non-null int64
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), int64(1), object(1)
memory usage: 7.2+ KB


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [92]:
y=pd.get_dummies(data,prefix=['Species'])
#data.drop(['Species'],axis=1, inplace=True)
y

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0
...,...,...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,0,0,1
146,147,6.3,2.5,5.0,1.9,0,0,1
147,148,6.5,3.0,5.2,2.0,0,0,1
148,149,6.2,3.4,5.4,2.3,0,0,1


In [105]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 3 columns):
Species_Iris-setosa        150 non-null uint8
Species_Iris-versicolor    150 non-null uint8
Species_Iris-virginica     150 non-null uint8
dtypes: uint8(3)
memory usage: 578.0 bytes


In [96]:
y.columns

Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species_Iris-setosa', 'Species_Iris-versicolor',
       'Species_Iris-virginica'],
      dtype='object')

In [0]:
y=y.drop('Id', axis=1)

In [0]:
y=y.drop('SepalLengthCm', axis=1)

In [0]:
y=y.drop('SepalWidthCm', axis=1)

In [0]:
y=y.drop('PetalLengthCm', axis=1)

In [0]:
y=y.drop('PetalWidthCm', axis=1)

In [102]:
y.columns

Index(['Species_Iris-setosa', 'Species_Iris-versicolor',
       'Species_Iris-virginica'],
      dtype='object')

In [103]:
X=data.drop('Species', axis=1)
X

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,1,5.1,3.5,1.4,0.2
1,2,4.9,3.0,1.4,0.2
2,3,4.7,3.2,1.3,0.2
3,4,4.6,3.1,1.5,0.2
4,5,5.0,3.6,1.4,0.2
...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3
146,147,6.3,2.5,5.0,1.9
147,148,6.5,3.0,5.2,2.0
148,149,6.2,3.4,5.4,2.3


In [138]:
X=X.drop('Id', axis=1)


KeyError: ignored

In [152]:
X

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


### Splitting the data into feature set and target set

In [0]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30)


In [155]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((105, 4), (45, 4), (105, 3), (45, 3))

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
model = tf.keras.Sequential()

In [0]:
#Add the Dense Layer for Prediction
model.add(tf.keras.layers.Dense(3, input_shape=(4,),activation="softmax"))

In [0]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

### Model Training 

In [159]:
model.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 3)                 15        
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


In [146]:
X_test.shape

(45, 4)

In [147]:
X_test.columns

Index(['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'], dtype='object')

In [148]:
y_test.columns

Index(['Species_Iris-setosa', 'Species_Iris-versicolor',
       'Species_Iris-virginica'],
      dtype='object')

In [149]:
y_test.shape

(45, 3)

In [0]:
#y_train = tf.keras.utils.to_categorical(y_train, num_classes=3)
#y_test = tf.keras.utils.to_categorical(y_test, num_classes=3)


In [160]:
model.fit(X_train,y_train,          
          validation_data=(X_test,y_test),
          epochs=100,
          batch_size=32)

Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 0s 1ms/sample - loss: 1.6176 - accuracy: 0.3524 - val_loss: 1.4444 - val_accuracy: 0.2889
Epoch 2/100
105/105 [==============================] - 0s 137us/sample - loss: 1.2167 - accuracy: 0.3524 - val_loss: 1.1079 - val_accuracy: 0.2889
Epoch 3/100
105/105 [==============================] - 0s 109us/sample - loss: 0.9873 - accuracy: 0.3619 - val_loss: 0.9422 - val_accuracy: 0.3111
Epoch 4/100
105/105 [==============================] - 0s 122us/sample - loss: 0.8437 - accuracy: 0.3714 - val_loss: 0.8437 - val_accuracy: 0.4444
Epoch 5/100
105/105 [==============================] - 0s 106us/sample - loss: 0.7642 - accuracy: 0.6476 - val_loss: 0.7990 - val_accuracy: 0.6000
Epoch 6/100
105/105 [==============================] - 0s 123us/sample - loss: 0.7152 - accuracy: 0.7048 - val_loss: 0.7469 - val_accuracy: 0.6222
Epoch 7/100
105/105 [==============================] - 0s 122us/sample - lo

### Model Prediction

In [161]:
prediction = model.predict(X_test[0:1])

In [162]:
prediction

array([[0.06926992, 0.44949874, 0.48123127]], dtype=float32)

In [163]:
y_test[0:1]


,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
86,0,1,0


### Save the Model

In [0]:
model.save('iris_model.h5')


### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?